In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [2]:
import requests
from mechafil import data

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datetime import datetime, timedelta, date

In [3]:
start_date = date(2021,3,15)
current_date = datetime.today().date() - timedelta(days=1)

def get_err(df, key):
    err = df['spacescope_' + key] - df['starboard_' + key]
    return err

STARBOARD='starboard'
SPACESCOPE='spacescope'

In [4]:
bp_starboard = data.query_historical_baseline_power(data_backend=STARBOARD)
bp_spacescope = data.query_historical_baseline_power(data_backend=SPACESCOPE)

rbp_starboard = data.query_historical_rb_power(data_backend=STARBOARD)
rbp_spacescope = data.query_historical_rb_power(data_backend=SPACESCOPE)

qap_starboard = data.query_historical_qa_power(data_backend=STARBOARD)
qap_spacescope = data.query_historical_qa_power(data_backend=SPACESCOPE)

In [5]:
plt.figure(figsize=(8,3))
plt.subplot(1,3,1)
err = bp_spacescope['baseline'] - bp_starboard['baseline']
plt.plot(bp_starboard['date'], bp_starboard['baseline'])
plt.plot(bp_spacescope['date'], bp_spacescope['baseline'], linestyle='--')
plt.xticks(rotation=45)
plt.title('Baseline\nError=%0.02f' % (np.sum(err),))

plt.subplot(1,3,2)
err = rbp_spacescope['rb_power'] - rbp_starboard['rb_power']
plt.plot(rbp_starboard['date'], rbp_starboard['rb_power'])
plt.plot(rbp_spacescope['date'], rbp_starboard['rb_power'], linestyle='--')
plt.xticks(rotation=45)
plt.title('RB Power\nError=%0.02f' % (np.sum(err),))

plt.subplot(1,3,3)
err = qap_starboard['qa_power'] - qap_starboard['qa_power']
plt.plot(qap_starboard['date'], qap_starboard['qa_power'])
plt.plot(qap_spacescope['date'], qap_starboard['qa_power'], linestyle='--')
plt.xticks(rotation=45)
plt.title('QA Power\nError=%0.02f' % (np.sum(err),))

plt.tight_layout()

<IPython.core.display.Javascript object>

In [6]:
onboarded_power_starboard = data.query_daily_power_onboarded(start_date, current_date, data_backend=STARBOARD)
onboarded_power_spacescope = data.query_daily_power_onboarded(start_date, current_date, data_backend=SPACESCOPE)

cols = onboarded_power_starboard.columns
new_name_mapping = {}
for c in cols:
    if 'date' not in c:
        new_name_mapping[c] = 'starboard_' + c
onboarded_power_starboard = onboarded_power_starboard.rename(columns=new_name_mapping)

cols = onboarded_power_spacescope.columns
new_name_mapping = {}
for c in cols:
    if 'date' not in c:
        new_name_mapping[c] = 'spacescope_' + c
onboarded_power_spacescope = onboarded_power_spacescope.rename(columns=new_name_mapping)

power_df_merged = onboarded_power_spacescope.merge(onboarded_power_starboard, on='date')

In [7]:
plt.figure(figsize=(6,3))

plt.subplot(1,2,1)
plt.plot(power_df_merged['date'], power_df_merged['starboard_day_onboarded_rb_power_pib'])
plt.plot(power_df_merged['date'], power_df_merged['spacescope_day_onboarded_rb_power_pib'], linestyle='--')
err = get_err(power_df_merged, 'day_onboarded_rb_power_pib')
plt.xticks(rotation=45)
plt.ylabel('PiB')
plt.title('RB Onboarded Power \n Error=%0.02f' % (np.sum(err),))

plt.subplot(1,2,2)
plt.plot(power_df_merged['date'], power_df_merged['starboard_day_onboarded_qa_power_pib'])
plt.plot(power_df_merged['date'], power_df_merged['spacescope_day_onboarded_qa_power_pib'], linestyle='--')
err = get_err(power_df_merged, 'day_onboarded_qa_power_pib')
plt.xticks(rotation=45)
plt.ylabel('PiB')
plt.title('QA Onboarded Power \n Error=%0.02f' % (np.sum(err),))

plt.tight_layout()

<IPython.core.display.Javascript object>

In [8]:
supply_starboard = data.query_supply_stats(start_date, current_date, data_backend=STARBOARD)
supply_spacescope = data.query_supply_stats(start_date, current_date, data_backend=SPACESCOPE)

# align dates to properly compute error
supply_starboard = supply_starboard.rename(
    columns={
        'circulating_fil': 'starboard_circulating_fil',
        'mined_fil': 'starboard_mined_fil',
        'vested_fil': 'starboard_vested_fil',
        'locked_fil': 'starboard_locked_fil',
        'burnt_fil': 'starboard_burnt_fil'
    }
)
supply_spacescope = supply_spacescope.rename(
    columns={
        'circulating_fil': 'spacescope_circulating_fil',
        'mined_fil': 'spacescope_mined_fil',
        'vested_fil': 'spacescope_vested_fil',
        'locked_fil': 'spacescope_locked_fil',
        'burnt_fil': 'spacescope_burnt_fil'
    }
)
supply_df_merged = supply_spacescope.merge(supply_starboard, on='date')

In [9]:
plt.figure(figsize=(8,6))

plt.subplot(2,3,1)
plt.plot(supply_df_merged['date'], supply_df_merged['starboard_circulating_fil'])
plt.plot(supply_df_merged['date'], supply_df_merged['spacescope_circulating_fil'], linestyle='--')
error = get_err(supply_df_merged, 'circulating_fil')
plt.xticks(rotation=45)
plt.ylabel('FIL')
plt.title('Circulating Supply \n Error=%0.02f' % (np.sum(error,)))

plt.subplot(2,3,2)
plt.plot(supply_df_merged['date'], supply_df_merged['starboard_mined_fil'])
plt.plot(supply_df_merged['date'], supply_df_merged['spacescope_mined_fil'], linestyle='--')
error = get_err(supply_df_merged, 'mined_fil')
plt.xticks(rotation=45)
plt.ylabel('FIL')
plt.title('Mined \n Error=%0.02f' % (np.sum(error,)))

plt.subplot(2,3,3)
plt.plot(supply_df_merged['date'], supply_df_merged['starboard_vested_fil'])
plt.plot(supply_df_merged['date'], supply_df_merged['spacescope_vested_fil'], linestyle='--')
error = get_err(supply_df_merged, 'vested_fil')
plt.xticks(rotation=45)
plt.ylabel('FIL')
plt.title('Vested \n Error=%0.02f' % (np.sum(error,)))

plt.subplot(2,3,4)
plt.plot(supply_df_merged['date'], supply_df_merged['starboard_locked_fil'])
plt.plot(supply_df_merged['date'], supply_df_merged['spacescope_locked_fil'], linestyle='--')
error = get_err(supply_df_merged, 'locked_fil')
plt.xticks(rotation=45)
plt.ylabel('FIL')
plt.title('Locked \n Error=%0.02f' % (np.sum(error,)))

plt.subplot(2,3,5)
plt.plot(supply_df_merged['date'], supply_df_merged['starboard_burnt_fil'])
plt.plot(supply_df_merged['date'], supply_df_merged['spacescope_burnt_fil'], linestyle='--')
error = get_err(supply_df_merged, 'burnt_fil')
plt.xticks(rotation=45)
plt.ylabel('FIL')
plt.title('Burnt \n Error=%0.02f' % (np.sum(error,)))

plt.tight_layout()

<IPython.core.display.Javascript object>

In [10]:
# test potential-expire by checking into the future
current_date_expire = current_date + timedelta(days=360)

starboard_scheduled_df = data.query_sector_expirations(start_date, current_date_expire, data_backend=STARBOARD)
spacescope_scheduled_df = data.query_sector_expirations(start_date, current_date_expire, data_backend=SPACESCOPE)

starboard_scheduled_df['extended_pledge'] = starboard_scheduled_df['extended_pledge'].astype(float)
starboard_scheduled_df['expired_pledge'] = starboard_scheduled_df['expired_pledge'].astype(float)
starboard_scheduled_df['potential_expire_pledge'] = starboard_scheduled_df['potential_expire_pledge'].astype(float)

cols = starboard_scheduled_df.columns
new_name_mapping = {}
for c in cols:
    if 'date' not in c:
        new_name_mapping[c] = 'starboard_' + c
starboard_scheduled_df = starboard_scheduled_df.rename(columns=new_name_mapping)

cols = spacescope_scheduled_df.columns
new_name_mapping = {}
for c in cols:
    if 'date' not in c:
        new_name_mapping[c] = 'spacescope_' + c
spacescope_scheduled_df = spacescope_scheduled_df.rename(columns=new_name_mapping)

scheduled_df_merged = spacescope_scheduled_df.merge(starboard_scheduled_df, on='date')

In [11]:
plt.figure(figsize=(9,6))

current_date_minus_one = current_date-timedelta(days=1)  # I guess this is how often the starboard backend is updated 
scheduled_df_merged_upto_today = scheduled_df_merged[scheduled_df_merged['date'] <= pd.to_datetime(current_date_minus_one, utc='UTC')]
scheduled_df_merged_future = scheduled_df_merged[scheduled_df_merged['date'] > pd.to_datetime(current_date_minus_one, utc='UTC')]

plt.subplot(3,3,1)
error = get_err(scheduled_df_merged, 'extended_rb')
if np.sum(error) > 0:
    plt.plot(scheduled_df_merged['date'], error)
else:
    plt.plot(scheduled_df_merged['date'], scheduled_df_merged['starboard_extended_rb'])
    plt.plot(scheduled_df_merged['date'], scheduled_df_merged['spacescope_extended_rb'], linestyle='--')    
plt.xticks(rotation=45)
plt.title('RB Extend \n Error=%0.02f' % (np.sum(error),))
plt.ylabel('PiB')

plt.subplot(3,3,2)
error = get_err(scheduled_df_merged_upto_today, 'expired_rb')
if np.sum(error) > 0:
    plt.plot(scheduled_df_merged_upto_today['date'], error)
else:
    plt.plot(scheduled_df_merged_upto_today['date'], scheduled_df_merged_upto_today['starboard_expired_rb'])
    plt.plot(scheduled_df_merged_upto_today['date'], scheduled_df_merged_upto_today['spacescope_expired_rb'], linestyle='--')    
plt.xticks(rotation=45)
plt.title('RB Expire \n Error=%0.02f' % (np.sum(error),))
plt.ylabel('PiB')

plt.subplot(3,3,3)
error = scheduled_df_merged_future['spacescope_expired_rb'] - scheduled_df_merged_future['starboard_open_rb']
if np.sum(error) > 0:
    plt.plot(scheduled_df_merged_future['date'], error)
else:
    plt.plot(scheduled_df_merged_future['date'], scheduled_df_merged_future['starboard_open_rb'])
    plt.plot(scheduled_df_merged_future['date'], scheduled_df_merged_future['spacescope_expired_rb'], linestyle='--')    
plt.xticks(rotation=45)
plt.title('RB Sched Expire [Future] \n Error=%0.02f' % (np.sum(error),))
plt.ylabel('PiB')

plt.subplot(3,3,4)
error = get_err(scheduled_df_merged, 'extended_qa')
if np.sum(error) > 0:
    plt.plot(scheduled_df_merged['date'], error)
else:
    plt.plot(scheduled_df_merged['date'], scheduled_df_merged['starboard_extended_qa'])
    plt.plot(scheduled_df_merged['date'], scheduled_df_merged['spacescope_extended_qa'], linestyle='--')    
plt.xticks(rotation=45)
plt.title('QA Extend \n Error=%0.02f' % (np.sum(error),))
plt.ylabel('PiB')

plt.subplot(3,3,5)
error = get_err(scheduled_df_merged_upto_today, 'expired_qa')
if np.sum(error) > 0:
    plt.plot(scheduled_df_merged_upto_today['date'], error)
else:
    plt.plot(scheduled_df_merged_upto_today['date'], scheduled_df_merged_upto_today['starboard_expired_qa'])
    plt.plot(scheduled_df_merged_upto_today['date'], scheduled_df_merged_upto_today['spacescope_expired_qa'], linestyle='--')    
plt.xticks(rotation=45)
plt.title('QA Expire \n Error=%0.02f' % (np.sum(error),))
plt.ylabel('PiB')

plt.subplot(3,3,6)
error = scheduled_df_merged_future['spacescope_expired_qa'] - scheduled_df_merged_future['starboard_open_qa']
if np.sum(error) > 0:
    plt.plot(scheduled_df_merged_future['date'], error)
else:
    plt.plot(scheduled_df_merged_future['date'], scheduled_df_merged_future['starboard_open_qa'])
    plt.plot(scheduled_df_merged_future['date'], scheduled_df_merged_future['spacescope_expired_qa'], linestyle='--')    
plt.xticks(rotation=45)
plt.title('QA Sched Expire [Future] \n Error=%0.02f' % (np.sum(error),))
plt.ylabel('PiB')

plt.subplot(3,3,7)
error = get_err(scheduled_df_merged, 'extended_pledge')
if np.sum(error) > 0:
    plt.plot(scheduled_df_merged['date'], error)
else:
    plt.plot(scheduled_df_merged['date'], scheduled_df_merged['starboard_extended_pledge'])
    plt.plot(scheduled_df_merged['date'], scheduled_df_merged['spacescope_extended_pledge'], linestyle='--')    
plt.xticks(rotation=45)
plt.title('Pledge Extend \n Error=%0.02f' % (np.sum(error),))
plt.ylabel('FIL')

plt.subplot(3,3,8)
error = get_err(scheduled_df_merged_upto_today, 'expired_pledge')
if np.sum(error) > 0:
    plt.plot(scheduled_df_merged_upto_today['date'], error)
else:
    plt.plot(scheduled_df_merged_upto_today['date'], scheduled_df_merged_upto_today['starboard_expired_pledge'])
    plt.plot(scheduled_df_merged_upto_today['date'], scheduled_df_merged_upto_today['spacescope_expired_pledge'], linestyle='--')    
plt.xticks(rotation=45)
plt.title('Pledge Expire \n Error=%0.02f' % (np.sum(error),))
plt.ylabel('FIL')

plt.subplot(3,3,9)
error = scheduled_df_merged_future['spacescope_expired_pledge'] - scheduled_df_merged_future['starboard_potential_expire_pledge']
if np.sum(error) > 0:
    plt.plot(scheduled_df_merged_future['date'], error)
else:
    plt.plot(scheduled_df_merged_future['date'], scheduled_df_merged_future['starboard_potential_expire_pledge'])
    plt.plot(scheduled_df_merged_future['date'], scheduled_df_merged_future['spacescope_expired_pledge'], linestyle='--')    
plt.xticks(rotation=45)
plt.title('Pledge Sched Expire [Future] \n Error=%0.02f' % (np.sum(error),))
plt.ylabel('FIL')

plt.tight_layout()

<IPython.core.display.Javascript object>

In [12]:
plt.figure(figsize=(6,6))
plt.subplot(3,1,1)

error = get_err(scheduled_df_merged, 'schedule_expire_rb')
if np.sum(error) > 0:
    plt.plot(scheduled_df_merged['date'], error)
else:
    plt.plot(scheduled_df_merged['date'], scheduled_df_merged['starboard_schedule_expire_rb'])
    plt.plot(scheduled_df_merged['date'], scheduled_df_merged['spacescope_schedule_expire_rb'], linestyle='--')    
plt.xticks(rotation=45)
plt.title('RB Sched Expire \n Error=%0.02f' % (np.sum(error),))
plt.ylabel('PiB')

plt.subplot(3,1,2)
error = get_err(scheduled_df_merged, 'schedule_expire_qa')
if np.sum(error) > 0:
    plt.plot(scheduled_df_merged['date'], error)
else:
    plt.plot(scheduled_df_merged['date'], scheduled_df_merged['starboard_schedule_expire_qa'])
    plt.plot(scheduled_df_merged['date'], scheduled_df_merged['spacescope_schedule_expire_qa'], linestyle='--')
plt.xticks(rotation=45)
plt.title('QA Sched Expire \n Error=%0.02f' % (np.sum(error),))
plt.ylabel('PiB')

plt.subplot(3,1,3)
error = get_err(scheduled_df_merged, 'schedule_expire_pledge')
if np.sum(error) > 0:
    plt.plot(scheduled_df_merged['date'], error)
else:
    plt.plot(scheduled_df_merged['date'], scheduled_df_merged['starboard_schedule_expire_pledge'])
    plt.plot(scheduled_df_merged['date'], scheduled_df_merged['spacescope_schedule_expire_pledge'], linestyle='--')
plt.xticks(rotation=45)
plt.title('Sched Expire Pledge \n Error=%0.02f' % (np.sum(error),))
plt.ylabel('FIL')

plt.tight_layout()

<IPython.core.display.Javascript object>

In [13]:
plt.figure(figsize=(6,6))
plt.subplot(3,1,1)

error = get_err(scheduled_df_merged, 'total_rb')
if np.sum(error) > 0:
    plt.plot(scheduled_df_merged['date'], error)
else:
    plt.plot(scheduled_df_merged['date'], scheduled_df_merged['starboard_total_rb'])
    plt.plot(scheduled_df_merged['date'], scheduled_df_merged['spacescope_total_rb'], linestyle='--')    
plt.xticks(rotation=45)
plt.title('Total RB Sched Expire \n Error=%0.02f' % (np.sum(error),))
plt.ylabel('PiB')

plt.subplot(3,1,2)
error = get_err(scheduled_df_merged, 'total_qa')
if np.sum(error) > 0:
    plt.plot(scheduled_df_merged['date'], error)
else:
    plt.plot(scheduled_df_merged['date'], scheduled_df_merged['starboard_total_qa'])
    plt.plot(scheduled_df_merged['date'], scheduled_df_merged['spacescope_total_qa'], linestyle='--')
plt.xticks(rotation=45)
plt.title('Total QA Sched Expire \n Error=%0.02f' % (np.sum(error),))
plt.ylabel('PiB')

plt.subplot(3,1,3)
error = get_err(scheduled_df_merged, 'total_pledge')
if np.sum(error) > 0:
    plt.plot(scheduled_df_merged['date'], error)
else:
    plt.plot(scheduled_df_merged['date'], scheduled_df_merged['starboard_total_pledge'])
    plt.plot(scheduled_df_merged['date'], scheduled_df_merged['spacescope_total_pledge'], linestyle='--')

plt.xticks(rotation=45)
plt.title('Total Pledge \n Error=%0.02f' % (np.sum(error),))
plt.ylabel('FIL')

plt.tight_layout()

<IPython.core.display.Javascript object>

In [14]:
# showcase typical usage to ensure those vectors are the same
forecast_length=365*4
end_date = current_date + timedelta(days=forecast_length)

starboard_fil_stats_df = data.get_historical_network_stats(start_date,current_date,end_date, data_backend=STARBOARD)
spacescope_fil_stats_df = data.get_historical_network_stats(start_date,current_date,end_date, data_backend=SPACESCOPE)

# merge to align dates
test_cols = []
cols = starboard_fil_stats_df.columns
new_name_mapping = {}
for c in cols:
    if 'date' not in c:
        new_name_mapping[c] = 'starboard_' + c
        test_cols.append(c)
starboard_fil_stats_df = starboard_fil_stats_df.rename(columns=new_name_mapping)

cols = spacescope_fil_stats_df.columns
new_name_mapping = {}
for c in cols:
    if 'date' not in c:
        new_name_mapping[c] = 'spacescope_' + c
spacescope_fil_stats_df = spacescope_fil_stats_df.rename(columns=new_name_mapping)

fil_stats_df_merged = spacescope_fil_stats_df.merge(starboard_fil_stats_df, on='date')

In [15]:
err_cols = []
for c in test_cols:
    spacescope_y = fil_stats_df_merged['spacescope_'+c]
    starboard_y = fil_stats_df_merged['starboard_'+c]
    err = spacescope_y-starboard_y
    if np.sum(err) != 0:
        err_cols.append((c, np.sum(err)))
nrow = len(err_cols)//3

plt.figure()
for ii,c in enumerate(err_cols):
    col_name = c[0]
    err = c[1]
    plt.subplot(3,nrow,ii+1)
    plt.plot(fil_stats_df_merged['date'], fil_stats_df_merged['starboard_'+col_name], label='starboard')
    plt.plot(fil_stats_df_merged['date'], fil_stats_df_merged['spacescope_'+col_name], label='spacescope', linestyle='--')
    plt.title(c)
plt.legend()
plt.tight_layout()

<IPython.core.display.Javascript object>

In [16]:
starboard_expire_stats = data.get_sector_expiration_stats(start_date, current_date, end_date, data_backend=STARBOARD)
spacescope_expire_stats = data.get_sector_expiration_stats(start_date, current_date, end_date, data_backend=SPACESCOPE)

starboard_rbp_expire, starboard_qap_expire, starboard_pledge_release = starboard_expire_stats
spacescope_rbp_expire, spacescope_qap_expire, spacescope_pledge_release = spacescope_expire_stats

print('RBP Expire Error=', np.sum(starboard_rbp_expire-spacescope_rbp_expire))
print('QAP Expire Error=', np.sum(starboard_qap_expire-spacescope_qap_expire))
print('Pledge Expire Error=', np.sum(starboard_pledge_release-spacescope_pledge_release))

RBP Expire Error= 0.0
QAP Expire Error= -1.0125233984581428e-13
Pledge Expire Error= 6.890314485641369e-09
